# Xác định thực thể trong câu bằng thư viện

## Xác định bằng thư viện Tiếng Anh

In [1]:
import spacy
nlp_en = spacy.load("en_core_web_sm")

text_en = "My name is Tuyen"

doc_en = nlp_en(text_en)

for ent in doc_en.ents:
    print(ent.text, ent.label_)


Tuyen ORG


## Xác định bằng thư viện Tiếng Việt

In [2]:
# import os
# import zipfile
# import requests

# url = "https://github.com/vncorenlp/VnCoreNLP/archive/v1.1.1.zip"
# zip_path = "VnCoreNLP-1.1.1.zip"

# if not os.path.exists(zip_path):
#     r = requests.get(url)
#     with open(zip_path, 'wb') as f:
#         f.write(r.content)

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall()


jar_path = "/root/code/Thuvien/VnCoreNLP-1.1.1/VnCoreNLP-1.1.1.jar"


In [3]:

from vncorenlp import VnCoreNLP

vncorenlp = VnCoreNLP(jar_path, annotators="wseg,pos,ner", max_heap_size='-Xmx2g')

def print_named_entities(text):
    annotations = vncorenlp.annotate(text)
    for sentence in annotations['sentences']:
        for token in sentence:
            if token['nerLabel'] != 'O':
                print(f"Từ: {token['form']}, POS: {token['posTag']}, NER: {token['nerLabel']}")

text_vi = "Hôm nay tôi đi học tại trường Nguyễn Trường Tộ"

print_named_entities(text_vi)


Từ: trường, POS: N, NER: B-ORG
Từ: Nguyễn_Trường_Tộ, POS: Np, NER: I-ORG


In [4]:
from vncorenlp import VnCoreNLP

vncorenlp = VnCoreNLP(jar_path, annotators="wseg,pos,ner", max_heap_size='-Xmx2g')

def print_named_entities(text):
    annotations = vncorenlp.annotate(text)
    for sentence in annotations['sentences']:
        for token in sentence:
            if token['nerLabel'] != 'O':
                print(f"Từ: {token['form']}, POS: {token['posTag']}, NER: {token['nerLabel']}")

text_vi = "Nguyễn Trường Tộ đang đi học tại Hà Nội"

print_named_entities(text_vi)


Từ: Nguyễn_Trường_Tộ, POS: Np, NER: B-PER
Từ: Hà_Nội, POS: Np, NER: B-LOC


# Xác định bằng thuật toán tự code, sử dụng cho Tiếng Việt MiAI

In [24]:
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import Input
from keras_contrib.layers import CRF
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import text_to_word_sequence

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import f1_score
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
import pickle
import os

from cls_sentence import sentence


ModuleNotFoundError: No module named 'keras_contrib'

In [22]:

df = pd.read_csv("./ner_dataset.csv", encoding = "ISO-8859-1")
df = df.fillna(method = 'ffill')
df.head()

/tmp/ipykernel_962/4021734431.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method = 'ffill')


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [29]:
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import Input
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import text_to_word_sequence

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import f1_score
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
import pickle
import os

from cls_sentence import sentence

# Config
batch_size = 64
epochs = 50
max_len = 75
embedding = 40

def load_data(filename='ner_dataset.csv'):
    df = pd.read_csv(filename, encoding="ISO-8859-1")
    df = df.fillna(method='ffill')
    return df

def process_data(df, sentences):
    words = list(df['Word'].unique())
    tags = list(df['Tag'].unique())

    word2idx = {w: i + 2 for i, w in enumerate(words)}
    word2idx["UNK"] = 1
    word2idx["PAD"] = 0

    tag2idx = {t: i + 1 for i, t in enumerate(tags)}
    tag2idx["PAD"] = 0

    idx2word = {i: w for w, i in word2idx.items()}
    idx2tag = {i: w for w, i in tag2idx.items()}

    X = [[word2idx[w[0]] for w in s] for s in sentences]
    X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=word2idx["PAD"])
    y = [[tag2idx[w[2]] for w in s] for s in sentences]
    y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["PAD"])

    num_tag = df['Tag'].nunique()
    y = [to_categorical(i, num_classes=num_tag + 1) for i in y]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

    return X_train, X_test, y_train, y_test, word2idx, tag2idx, idx2word, idx2tag, num_tag, words, tags

def build_model(num_tags, hidden_size=50):
    input = Input(shape=(max_len,))
    model = Embedding(input_dim=len(words) + 2, output_dim=embedding, input_length=max_len, mask_zero=False)(input)
    model = Bidirectional(LSTM(units=hidden_size, return_sequences=True, recurrent_dropout=0.1))(model)
    model = TimeDistributed(Dense(hidden_size, activation="relu"))(model)
    out = TimeDistributed(Dense(num_tags + 1, activation="softmax"))(model)  # Thay thế CRF bằng lớp Dense

    model = Model(input, out)
    model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

    model.summary()
    return model

if not os.path.exists("data.pkl"):
    print("Data not found, make it!")
    df = load_data()
    getter = sentence(df)
    sentences = getter.sentences
    X_train, X_test, y_train, y_test, word2idx, tag2idx, idx2word, idx2tag, num_tag, words, tags = process_data(df, sentences)
    with open('data.pkl', 'wb') as file:
        data = [X_train, X_test, y_train, y_test, word2idx, tag2idx, idx2word, idx2tag, num_tag, words, tags]
        pickle.dump(data, file)
else:
    with open("data.pkl", "rb") as f:
        data = pickle.load(f)
        X_train, X_test, y_train, y_test, word2idx, tag2idx, idx2word, idx2tag, num_tag, words, tags = data

if not os.path.exists("model.keras"):
    model = build_model(num_tag)
    checkpoint = ModelCheckpoint(filepath='model.keras',
                                 verbose=0,
                                 mode='auto',
                                 save_best_only=True,
                                 monitor='val_loss')
    history = model.fit(X_train, np.array(y_train), batch_size=batch_size, epochs=epochs,
                        validation_split=0.1, callbacks=[checkpoint])
else:
    model = build_model(num_tag)
    model.load_weights("model.keras")

y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=-1)
y_test_true = np.argmax(y_test, -1)

y_pred = [[idx2tag[i] for i in row] for row in y_pred]
y_test_true = [[idx2tag[i] for i in row] for row in y_test_true]
print("F1-score is : {:.1%}".format(f1_score(y_test_true, y_pred)))

idx = np.random.randint(0, X_test.shape[0])

p = model.predict(np.array([X_test[idx]]))
p = np.argmax(p, axis=-1)
true = np.argmax(y_test[idx], -1)

print("Example #{}".format(idx))

print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(40 * "*")
for w, t, pred in zip(X_test[idx], true, p[0]):
    if w != 0:
        print("{:15}: {:5} {}".format(words[w-2], idx2tag[t], idx2tag[pred]))


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 75)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ (None, 75, 40)         │     1,407,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 75, 100)        │        36,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 75, 50)         │         5,050 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 75, 18)         │           918 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,449,528 (5.53 MB)

 Trainable params: 1,449,528 (5.53 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50


2024-06-07 23:04:49.350953: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 198115200 exceeds 10% of free system memory.


574/574 ━━━━━━━━━━━━━━━━━━━━ 78s 121ms/step - accuracy: 0.9388 - loss: 0.3295 - val_accuracy: 0.9638 - val_loss: 0.1289
Epoch 2/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 69s 120ms/step - accuracy: 0.9671 - loss: 0.1139 - val_accuracy: 0.9750 - val_loss: 0.0880
Epoch 3/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 68s 118ms/step - accuracy: 0.9783 - loss: 0.0788 - val_accuracy: 0.9835 - val_loss: 0.0631
Epoch 4/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 72s 125ms/step - accuracy: 0.9851 - loss: 0.0556 - val_accuracy: 0.9861 - val_loss: 0.0506
Epoch 5/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 70s 122ms/step - accuracy: 0.9881 - loss: 0.0424 - val_accuracy: 0.9875 - val_loss: 0.0435
Epoch 6/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 80s 139ms/step - accuracy: 0.9895 - loss: 0.0366 - val_accuracy: 0.9885 - val_loss: 0.0400
Epoch 7/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 68s 118ms/step - accuracy: 0.9906 - loss: 0.0324 - val_accuracy: 0.9885 - val_loss: 0.0413
Epoch 8/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 68s 119ms/step - accuracy: 0.9914 - loss: 0.0297 - val

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PAD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


F1-score is : 85.1%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Example #1681
Word           ||True ||Pred
****************************************
The            : O     O
Israeli        : B-gpe B-gpe
military       : O     O
said           : O     O
soldiers       : O     O
have           : O     O
uncovered      : O     O
at             : O     O
least          : O     O
three          : O     O
explosives     : O     O
laboratories   : O     O
.              : O     O


In [31]:
# Test với một câu ngẫu nhiên trong tập test
idx = np.random.randint(0,X_test.shape[0])

p = model.predict(np.array([X_test[idx]]))
p = np.argmax(p, axis=-1)
true = np.argmax(y_test[idx], -1)

print("Example #{}".format(idx))

print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(40 * "*")
for w, t, pred in zip(X_test[idx], true, p[0]):
    if w != 0:
        print("{:15}: {:5} {}".format(words[w-2], idx2tag[t], idx2tag[pred]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Example #6078
Word           ||True ||Pred
****************************************
The            : O     O
Athens         : B-org B-org
Geodynamic     : I-org I-org
Institute      : I-org I-org
says           : O     O
Friday         : B-tim B-tim
's             : O     O
magnitude      : O     O
5.1            : O     O
quake          : O     O
was            : O     O
centered       : O     O
near           : O     O
the            : O     O
eastern        : O     O
islands        : O     O
of             : O     O
Kos            : B-geo B-geo
and            : O     O
Astypaleia     : B-geo B-geo
,              : O     O
about          : O     O
300            : O     O
kilometers     : O     O
from           : O     O
the            : O     O
Greek          : B-gpe B-gpe
capital        : O     O
.              : O     O
